In [1]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
# Import all required libraries
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
!pip install emoji==1.7.0
import emoji
import numpy as np
import re
import string

Mounted at /content/drive


# Data Preprocessing

## Loading Dataframe


In [22]:


# Load the Excel file into a DataFrame
df = pd.read_excel('/content/drive/MyDrive/Subasish(2023)_Twitter/Tutorial/15minutecity.xlsx')

In [23]:
df.columns

Index(['tweet_id', 'user_username', 'text', 'created_at', 'date',
       'conversation_id', 'possibly_sensitive', 'author_id', 'lang',
       'in_reply_to_user_id', 'user_name', 'user_verified', 'user_location',
       'user_created_at', 'user_protected', 'user_profile_image_url',
       'user_description', 'user_pinned_tweet_id', 'user_url', 'retweet_count',
       'like_count', 'quote_count', 'user_tweet_count', 'user_list_count',
       'user_followers_count', 'user_following_count', 'sourcetweet_type',
       'sourcetweet_id', 'sourcetweet_text', 'sourcetweet_lang',
       'sourcetweet_author_id'],
      dtype='object')

In [24]:
# Extract specific columns
selected_columns = df[['tweet_id','text', 'sourcetweet_type', 'sourcetweet_id']]
# Display the extracted columns
print(selected_columns.head(5))


              tweet_id  \
0  1024126649057590016   
1  1024038782842986496   
2  1023921603334287361   
3  1023920796153069569   
4  1023700403945439232   

                                                                                                                                                                                                                                                                                       text  \
0                                                                                                                                              RT @erinriediger: Another example of how cities can be re-designed for people even if we've made urban planning mistakes in the past. The c…   
1                       #KNOWYOUREDMO\nAny idea where this spot is in town? Take a guess.\n\n#guessinggame #knowyourcity #deepedmonton #yeg #edmo #edmonton #walkyeg #yegwalk #walkable #walkableyeg #walkableedmonton #walking #walkability #alberta #walkablecity https://t

## Find original tweet

Retweet


In [32]:

pd.set_option('display.max_colwidth', None)
print(selected_columns['text'].iloc[47])


#Winnipeg was a multimodal city not that long ago. A rare colour photo of street cars on Main Street. #walkablecity #citiesforpeople https://t.co/clz4ybAwaO


Original

In [29]:
print(selected_columns['text'].iloc[7])

RT @erinriediger: #Winnipeg was a multimodal city not that long ago. A rare colour photo of street cars on Main Street. #walkablecity #citi…


In [33]:

# Define a lambda function to convert the value to fully expanded form
convert_to_expanded_form = lambda x: '{:f}'.format(x)

# Apply the lambda function to the 'sourcetweet_id' column
selected_columns['sourcetweet_id'] = selected_columns['sourcetweet_id'].apply(convert_to_expanded_form)

# Print the updated DataFrame
print(selected_columns['sourcetweet_id'])


0         1023348768626941952.000000
1                                nan
2                                nan
3                                nan
4                                nan
                     ...            
139250     986130215045943296.000000
139251     986130215045943296.000000
139252                           nan
139253     986130215045943296.000000
139254     986130215045943296.000000
Name: sourcetweet_id, Length: 139255, dtype: object


In [34]:

# Remove the last three digits and any periods from the 'tweet_id' column
selected_columns['tweet_id'] = selected_columns['tweet_id'].apply(lambda x: str(x).replace('.', '')[:-3].zfill(9))

# Remove the last three digits and any periods from the 'sourcetweet_id' column
selected_columns['sourcetweet_id'] = selected_columns['sourcetweet_id'].apply(lambda x: str(x).replace('.', '')[:-3].zfill(9))

# Print the updated DataFrame
print(selected_columns[['tweet_id', 'sourcetweet_id']])


                tweet_id          sourcetweet_id
0       1024126649057590  1023348768626941952000
1       1024038782842986               000000000
2       1023921603334287               000000000
3       1023920796153069               000000000
4       1023700403945439               000000000
...                  ...                     ...
139250   986261125779152   986130215045943296000
139251   986255604439187   986130215045943296000
139252   986254894951796               000000000
139253   986244086553165   986130215045943296000
139254   986238321650089   986130215045943296000

[139255 rows x 2 columns]


In [35]:

selected_columns['sourcetweet_id'] = selected_columns['sourcetweet_id'].apply(lambda x: str(x).replace('.', '')[:-6].zfill(9))

# Fetch the value at the specified row (row 46 in this case)
print(selected_columns['tweet_id'].iloc[46])

print(selected_columns['sourcetweet_id'].iloc[46])
print(selected_columns['tweet_id'].iloc[47])


1022668093141786
1022659798632939
1022659798632939


In [36]:

# Find the value in the 'sourcetweet_id' column at index 46
sourcetweet_id_value = selected_columns['sourcetweet_id'].iloc[46]

# Check all rows of the 'tweet_id' column for the sourcetweet_id_value
matching_rows = selected_columns[selected_columns['tweet_id'] == sourcetweet_id_value]

# Check if any matching rows were found
if not matching_rows.empty:
    # Store the tweet_id from the first matching row as an integer
    tweet_id_value = int(matching_rows['tweet_id'].iloc[0])
    print(tweet_id_value)
else:
    print("No rows found with the specified sourcetweet_id_value.")


1022659798632939


takes around 20 min


In [ ]:


# Calculate the number of rows for the first quarter
quarter_size = len(selected_columns) // 4

# Take the first quarter of the DataFrame
clean_final_1 = selected_columns.head(quarter_size).copy()

# Find the 'tweet_id' values of the source tweets in the clean_final_1 DataFrame
source_tweet_ids = clean_final_1.loc[
    clean_final_1['sourcetweet_type'].isin(['retweeted', 'quoted']),
    'sourcetweet_id'
].tolist()

# Iterate over the source tweet ids
for source_tweet_id in source_tweet_ids:
    # Find the row indices corresponding to the source tweet_id in the clean_final DataFrame
    source_indices = clean_final_1.loc[
        (clean_final_1['sourcetweet_type'].isin(['retweeted', 'quoted'])) &
        (clean_final_1['sourcetweet_id'] == source_tweet_id)
    ].index

    # Check if any source rows exist
    if len(source_indices) > 0:
        # Check if there is at least one matching row in the clean_final DataFrame
        if clean_final_1.loc[clean_final_1['tweet_id'] == source_tweet_id].shape[0] > 0:
            # Retrieve the text from the source row in the clean_final DataFrame
            text_value = clean_final_1.loc[clean_final_1['tweet_id'] == source_tweet_id, 'text'].values[0]

            # Update the 'text1' column for rows considered retweets in the clean_final DataFrame
            clean_final_1.loc[source_indices, 'text1'] = text_value

# Display the updated clean_final DataFrame
print(clean_final_1)





In [ ]:

# Calculate the number of rows for each quarter
quarter_size = len(df) // 4

# Take the second quarter of the DataFrame
clean_final_2 = df.iloc[quarter_size:2 * quarter_size].copy()

# Find the 'tweet_id' values of the source tweets in the clean_final DataFrame
source_tweet_ids = clean_final_2.loc[
    clean_final_2['sourcetweet_type'].isin(['retweeted', 'quoted']),
    'sourcetweet_id'
].tolist()

# Iterate over the source tweet ids
for source_tweet_id in source_tweet_ids:
    # Find the row indices corresponding to the source tweet_id in the clean_final DataFrame
    source_indices = clean_final_2.loc[
        (clean_final_2['sourcetweet_type'].isin(['retweeted', 'quoted'])) &
        (clean_final_2['sourcetweet_id'] == source_tweet_id)
    ].index

    # Check if any source rows exist
    if len(source_indices) > 0:
        # Check if there is at least one matching row in the clean_final DataFrame
        if clean_final_2.loc[clean_final_2['tweet_id'] == source_tweet_id].shape[0] > 0:
            # Retrieve the text from the source row in the clean_final DataFrame
            text_value = clean_final_2.loc[clean_final_2['tweet_id'] == source_tweet_id, 'text'].values[0]

            # Update the 'text1' column for rows considered retweets in the clean_final DataFrame
            clean_final_2.loc[source_indices, 'text1'] = text_value

# Display the updated clean_final DataFrame
print(clean_final_2)


In [ ]:

# Calculate the number of rows for each quarter
quarter_size = len(df) // 4

# Take the third quarter of the DataFrame
clean_final_3 = df.iloc[2 * quarter_size:3 * quarter_size].copy()

# Find the 'tweet_id' values of the source tweets in the clean_final DataFrame
source_tweet_ids = clean_final_3.loc[
    clean_final_3['sourcetweet_type'].isin(['retweeted', 'quoted']),
    'sourcetweet_id'
].tolist()

# Iterate over the source tweet ids
for source_tweet_id in source_tweet_ids:
    # Find the row indices corresponding to the source tweet_id in the clean_final DataFrame
    source_indices = clean_final_3.loc[
        (clean_final_3['sourcetweet_type'].isin(['retweeted', 'quoted'])) &
        (clean_final_3['sourcetweet_id'] == source_tweet_id)
    ].index

    # Check if any source rows exist
    if len(source_indices) > 0:
        # Check if there is at least one matching row in the clean_final DataFrame
        if clean_final_3.loc[clean_final_3['tweet_id'] == source_tweet_id].shape[0] > 0:
            # Retrieve the text from the source row in the clean_final DataFrame
            text_value = clean_final_3.loc[clean_final_3['tweet_id'] == source_tweet_id, 'text'].values[0]

            # Update the 'text1' column for rows considered retweets in the clean_final DataFrame
            clean_final_3.loc[source_indices, 'text1'] = text_value

# Display the updated clean_final DataFrame
print(clean_final_3)


In [ ]:

# Calculate the number of rows for each quarter
quarter_size = len(df) // 4

# Take the fourth quarter of the DataFrame (which will result in the entire DataFrame)
clean_final_4 = df.iloc[3 * quarter_size:].copy()

# Find the 'tweet_id' values of the source tweets in the clean_final DataFrame
source_tweet_ids = clean_final_4.loc[
    clean_final_4['sourcetweet_type'].isin(['retweeted', 'quoted']),
    'sourcetweet_id'
].tolist()

# Iterate over the source tweet ids
for source_tweet_id in source_tweet_ids:
    # Find the row indices corresponding to the source tweet_id in the clean_final DataFrame
    source_indices = clean_final_4.loc[
        (clean_final_4['sourcetweet_type'].isin(['retweeted', 'quoted'])) &
        (clean_final_4['sourcetweet_id'] == source_tweet_id)
    ].index

    # Check if any source rows exist
    if len(source_indices) > 0:
        # Check if there is at least one matching row in the clean_final DataFrame
        if clean_final_4.loc[clean_final_4['tweet_id'] == source_tweet_id].shape[0] > 0:
            # Retrieve the text from the source row in the clean_final DataFrame
            text_value = clean_final_4.loc[clean_final_4['tweet_id'] == source_tweet_id, 'text'].values[0]

            # Update the 'text1' column for rows considered retweets in the clean_final DataFrame
            clean_final_4.loc[source_indices, 'text1'] = text_value

# Display the updated clean_final DataFrame
print(clean_final_4)


Check known retweet and source tweet. They should be the same

In [ ]:

# Concatenate the DataFrames in numerical order
combined_df = pd.concat([clean_final_1, clean_final_2, clean_final_3, clean_final_4], ignore_index=True)

# Display the combined DataFrame
print(combined_df)



In [ ]:
print(len(combined_df))

In [ ]:
#retweet
combined_df['text1'].iloc[7]

In [ ]:
combined_df['text1'].iloc[47]

## Text Cleaning

Copy text from original tweets to text1 column

In [ ]:


# Assuming you already have the DataFrame named selected_columns

# Find the rows with 'sourcetweet_type' equal to NaN and not classified as retweet or quoted
nan_rows = combined_df[(combined_df['sourcetweet_type'].isna()) &
                            (~combined_df['sourcetweet_type'].isin(['retweeted', 'quoted']))]

# Copy the contents from 'text' to 'text1' for the identified rows
combined_df.loc[nan_rows.index, 'text1'] = nan_rows['text']

# Display the updated DataFrame
print(combined_df)


Create mention column

In [46]:

import pandas as pd


# Assuming you have the DataFrame 'df' with the 'text1' column
combined_df['mention'] = combined_df['text1'].str.findall(r'@(\w+)\s').apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Replace NaN values with an empty string
combined_df['mention'] = combined_df['mention'].replace(np.nan, '')

# Display the updated DataFrame
print(combined_df['mention'].head(10))



0                                                                
1                                                                
2    JaniceLukes, OlyBackstrom, pensato, adamdooley, 880CitiesOrg
3                                                                
4                                                                
5                                                                
6                                                                
7                                                                
8                                                                
9                                                                
Name: mention, dtype: object


Create hashtag column

In [47]:

# Assuming you have the DataFrame 'df' with the 'text1' column
combined_df['hashtag'] = combined_df['text1'].str.findall(r'#(\w+)\s').apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Replace NaN values with an empty string
combined_df['hashtag'] = combined_df['hashtag'].replace(np.nan, '')

# Display the updated DataFrame
print(combined_df['hashtag'])

0                                                                                                                            PortageAndMain, TeamOpen, wpgpoli, citiesforpeople, walkablecity
1         KNOWYOUREDMO, guessinggame, knowyourcity, deepedmonton, yeg, edmo, edmonton, walkyeg, yegwalk, walkable, walkableyeg, walkableedmonton, walking, walkability, alberta, walkablecity
2                                                                                                                                                                                walkablecity
3                                                                                                                                                            walkablecity, safecity, teamopen
4                                                                   pride, vancouver, vancouverbc, vancouverpride, helmckenstreet, burrard, burrardstreet, aptasustainability, apta, walkable
                                                  

create links column

In [48]:


# Assuming you have the DataFrame 'df' with the 'text1' column
combined_df['links'] = combined_df['text1'].apply(lambda x: ', '.join(re.findall(r'(https?://\S+)', str(x))))

# Display the updated DataFrame
print(combined_df['links'])




0         https://t.co/ZDixI3ku0m
1         https://t.co/fRbetWbNsa
2                                
3         https://t.co/75pCVt9Lt3
4         https://t.co/viYI80saXO
                   ...           
139250    https://t.co/eOCcthVOWR
139251    https://t.co/eOCcthVOWR
139252    https://t.co/7Ag6O4XyV5
139253    https://t.co/eOCcthVOWR
139254    https://t.co/eOCcthVOWR
Name: links, Length: 139255, dtype: object


Create Emoji column

In [49]:



def extract_emojis(s):
    if pd.isna(s):
        return ''
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

combined_df['emoji'] = combined_df['text1'].apply(extract_emojis)

# Display the updated DataFrame
print(combined_df['emoji'])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=0df273d446875006b38696955176face742c3c2721b136ab8316f8e926f828cb
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
0          
1          
2          
3          
4         🌈
         ..
139250     
139251     
139252     
139253     
139254     
Name: emoji, Length: 139255, dtype: object


In [50]:




# Assuming you have the DataFrame 'combined_df' with the 'text1' column

def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

def clean_text(s):
    if isinstance(s, str):
        # Remove mentions
        s = re.sub(r'@\w+\s?', '', s)
        # Remove hashtags
        s = re.sub(r'#\w+\s?', '', s)
        # Remove links
        s = re.sub(r'http\S+', '', s)
        # Remove emojis
        s = re.sub(emoji.get_emoji_regexp(), '', s)
        # Remove punctuation
        s = s.translate(str.maketrans('', '', string.punctuation))
        # Remove extra spaces
        s = re.sub(r'\s+', ' ', s).strip()
        # Remove "RT :" prefix
        s = re.sub(r'RT :', '', s).strip()
    return s

combined_df['text_clean'] = combined_df['text1'].apply(clean_text)

# Display the updated DataFrame
print(combined_df['text_clean'].head(5))


0                          Another example of how cities can be redesigned for people even if weve made urban planning mistakes in the past The comfortable pedestrian environment now coexists with car lanes
1                                                                                                                                                             Any idea where this spot is in town Take a guess
2    If you support and accessibility does that mean you support opening the intersection to pedestrians Even if you think the public consult process was lacking it seems your views would be supportive of a
3                                                                                                                                                                               Remove barriers coalition says
4                                                                                                                                                                           

In [51]:

# Assuming you have the DataFrame 'df' with the 'text_clean' column
combined_df['text_clean'] = combined_df['text_clean'].str.replace(r'\s+', ' ')

combined_df['text_clean'] = combined_df['text_clean'].str.lower()

combined_df['text_clean'] = combined_df['text_clean'].str.replace(r"[—“,.:\";=()%’/•\d\\]", "")

# Display the updated DataFrame
print(combined_df['text_clean'])

0                               another example of how cities can be redesigned for people even if weve made urban planning mistakes in the past the comfortable pedestrian environment now coexists with car lanes
1                                                                                                                                                                  any idea where this spot is in town take a guess
2         if you support and accessibility does that mean you support opening the intersection to pedestrians even if you think the public consult process was lacking it seems your views would be supportive of a
3                                                                                                                                                                                    remove barriers coalition says
4                                                                                                                                                       

In [54]:
clean_data = combined_df['text_clean']
#print(clean_data)
# Display the DataFrame columns
print(combined_df.columns)
#print(combined_df['tweet_id'])

Index(['tweet_id', 'text', 'sourcetweet_type', 'sourcetweet_id', 'text1',
       'user_username', 'created_at', 'date', 'conversation_id',
       'possibly_sensitive', 'author_id', 'lang', 'in_reply_to_user_id',
       'user_name', 'user_verified', 'user_location', 'user_created_at',
       'user_protected', 'user_profile_image_url', 'user_description',
       'user_pinned_tweet_id', 'user_url', 'retweet_count', 'like_count',
       'quote_count', 'user_tweet_count', 'user_list_count',
       'user_followers_count', 'user_following_count', 'sourcetweet_text',
       'sourcetweet_lang', 'sourcetweet_author_id', 'mention', 'hashtag',
       'links', 'emoji', 'text_clean'],
      dtype='object')


## Handle any tweets with sourcetweet_id's outside of each quartile

In [ ]:

# Get the tweet_id of the row at index 50 in the 'text1' column
source_tweet_id = combined_df['sourcetweet_id'].iloc[50]

# Find the row with the matching tweet_id
source_row = combined_df[combined_df['tweet_id'] == source_tweet_id]

# Check if a matching row is found
if not source_row.empty:
    # Get the text value from the source row
    text_value = source_row.iloc[0]['text']

    # Update the 'text1' column in the original row
    combined_df.at[50, 'text1'] = text_value

# Display the updated merged DataFrame
print(combined_df)


In [56]:
#print(clean_final.iloc[50])
empty_text1_count = combined_df['text1'].isnull().sum()

# Display the count
print("Number of rows with empty 'text1':", empty_text1_count)

Number of rows with empty 'text1': 24813


In [57]:

# Find the rows with an empty 'text1' column
empty_text1_rows = combined_df[combined_df['text1'].isnull()]

# Iterate over the empty_text1_rows
for index, row in empty_text1_rows.iterrows():
    # Get the 'tweet_id' from the current row
    tweet_id = row['tweet_id']

    # Find the corresponding 'sourcetweet_id'
    sourcetweet_id = combined_df[combined_df['tweet_id'] == tweet_id]['sourcetweet_id'].iloc[0]

    # Find the row with the matching 'tweet_id' and update 'text1' with the 'text' value********
    matching_row = combined_df[combined_df['tweet_id'] == sourcetweet_id]
    if not matching_row.empty:
        text_value = matching_row.iloc[0]['text']
        combined_df.loc[index, 'text1'] = text_value

# Print the updated 'text1' values
print(combined_df['text1'].iloc[50])


nan


In [58]:
# Assuming you have the DataFrame 'df' with the 'text1' column
combined_df['mention'] = combined_df['text1'].str.findall(r'@(\w+)\s').apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Replace NaN values with an empty string
combined_df['mention'] = combined_df['mention'].replace(np.nan, '')

# Assuming you have the DataFrame 'df' with the 'text1' column
combined_df['hashtag'] = combined_df['text1'].str.findall(r'#(\w+)\s').apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Replace NaN values with an empty string
combined_df['hashtag'] = combined_df['hashtag'].replace(np.nan, '')

# Assuming you have the DataFrame 'df' with the 'text1' column
combined_df['hashtag'] = combined_df['text1'].str.findall(r'#(\w+)\s').apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Replace NaN values with an empty string
combined_df['hashtag'] = combined_df['hashtag'].replace(np.nan, '')

!pip install emoji==1.7.0
import emoji

# Assuming you have the DataFrame 'selected_columns' with the 'text1' column
def extract_emojis(s):
    if pd.isna(s):
        return ''
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

combined_df['emoji'] = combined_df['text1'].apply(extract_emojis)


In [71]:
print(combined_df.iloc[50])

tweet_id                                                                                                                                 1020238659096891
text                      RT @Voetgangersbond: Inspirerend voorbeeld van voetgangersbeleid in Londen #walkablecities #voetgangers https://t.co/hUv4dhwu4y
sourcetweet_type                                                                                                                                retweeted
sourcetweet_id                                                                                                                           1020200582064287
text1                                          Inspirerend voorbeeld van voetgangersbeleid in Londen #walkablecities #voetgangers https://t.co/hUv4dhwu4y
user_username                                                                                                                                         NaN
created_at                                                                  

In [ ]:


# Drop rows with empty 'text1' column
combined_df.dropna(subset=['text1'], inplace=True)

# Print the updated DataFrame
print(combined_df)


In [61]:


# Save the merged DataFrame as an Excel file
combined_df.to_excel('/content/drive/MyDrive/Subasish(2023)_Twitter/Tutorial/clean_15minutecity.xlsx', index=False)

print("Excel file saved successfully.")


Excel file saved successfully.


## Translation
Save original language to a new df column

https://github.com/ssut/py-googletrans

In [62]:
df_clean = pd.read_excel('/content/drive/MyDrive/Subasish(2023)_Twitter/Tutorial/clean_15minutecity.xlsx')

In [ ]:
non_english_rows = df_clean[df_clean['lang'] != 'en']
row_numbers = non_english_rows.index.tolist()

if len(row_numbers) > 0:
    print("Rows with non-English language detected:")
    for row_number in row_numbers:
        print(row_number)
else:
    print("All rows have English language (en).")




In [64]:
# Assuming you want to print the third row (position 2)
print(df_clean['text_clean'].iloc[50])


inspirerend voorbeeld van voetgangersbeleid in londen


In [65]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=3f3a18efa6d31a6aea64581f372d030424c5af7b60abdaa3858e671733e83385
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

In [66]:
non_english_rows = []

# Iterate over each row
for index, row in df_clean.iterrows():
    if row['lang'] != 'en':
        non_english_rows.append(row['tweet_id'])

# Print the list of tweet_ids for non-English rows
#print(non_english_rows)

# Print the length of non_english_rows
print(len(non_english_rows))
print(len(df_clean))

38404
115248


In [67]:
'''
https://github.com/ssut/py-googletrans
'''

from googletrans import Translator

translator = Translator(service_urls=['translate.googleapis.com'])

# Iterate over the non-English rows
for tweet_id in non_english_rows:
    # Get the text to be translated based on tweet_id
    text = df_clean.loc[df_clean['tweet_id'] == tweet_id, 'text_clean'].iloc[0]

    # Translate the text
    translation = translator.translate(text)
    translated_text = translation.text

    # Update the DataFrame with the translated text
    df_clean.loc[df_clean['tweet_id'] == tweet_id, 'text_clean'] = translated_text
    df_clean.loc[df_clean['tweet_id'] == tweet_id, 'original_language'] = df_clean.loc[df_clean['tweet_id'] == tweet_id, 'lang'].iloc[0]

# Display the updated DataFrame
print(df_clean[['text_clean', 'original_language']])




                                                                                                                                                                                                       text_clean  \
0                            another example of how cities can be redesigned for people even if we've made urban planning mistakes in the past the comfortable pedestrian environment now coexists with car lanes   
1                                                                                                                                                                any idea where this spot is in town take a guess   
2       if you support and accessibility does that mean you support opening the intersection to pedestrians even if you think the public consult process was lacking it seems your views would be supportive of a   
3                                                                                                                                                   

### Check to see if translation worked.

In [68]:
print(df_clean.iloc[50])

tweet_id                                                                                                                                 1020238659096891
text                      RT @Voetgangersbond: Inspirerend voorbeeld van voetgangersbeleid in Londen #walkablecities #voetgangers https://t.co/hUv4dhwu4y
sourcetweet_type                                                                                                                                retweeted
sourcetweet_id                                                                                                                         1020200582064287.0
text1                                          Inspirerend voorbeeld van voetgangersbeleid in Londen #walkablecities #voetgangers https://t.co/hUv4dhwu4y
user_username                                                                                                                                         NaN
created_at                                                                  

In [70]:
df_clean['text_clean'] = df_clean['text_clean'].str.lower()
df_clean['text_clean'] = df_clean['text_clean'].str.replace('[\'—,’.“”)(!?…"]', '', regex=True)
df_clean['text_clean'] = df_clean['text_clean'].str.replace(r"[—“,.:\";=()%/•\d\\]", "")
df_clean['text_clean']

0                               another example of how cities can be redesigned for people even if weve made urban planning mistakes in the past the comfortable pedestrian environment now coexists with car lanes
1                                                                                                                                                                  any idea where this spot is in town take a guess
2         if you support and accessibility does that mean you support opening the intersection to pedestrians even if you think the public consult process was lacking it seems your views would be supportive of a
3                                                                                                                                                                                    remove barriers coalition says
4                                                                                                                                                       

In [72]:

# Save the updated DataFrame to a new .xlsx file
output_file = "/content/drive/MyDrive/Subasish(2023)_Twitter/Tutorial/translated_15minutecity.xlsx"
df_clean.to_excel(output_file, index=False)


In [ ]:
#df_clean('/content/drive/MyDrive/Subasish(2023)_Twitter/AV', index = False)

#find unique tweets

In [73]:

df_clean = pd.read_excel('/content/drive/MyDrive/Subasish(2023)_Twitter/Tutorial/translated_15minutecity.xlsx')

# Combine the two DataFrames
clean_df = pd.concat([df_clean], ignore_index=True)

In [74]:
len(clean_df)

115248

In [ ]:


# Assuming df is your DataFrame
num_rows = len(clean_df)
start_number = 0  # Change this if you want to start from a different number

# Generate the list of verif_ID values
verif_id_list = [f'a{i:08d}' for i in range(start_number, start_number + num_rows)]

# Insert the 'verif_ID' column at the beginning of the DataFrame
clean_df.insert(0, 'verif_ID', verif_id_list)

# Print the updated DataFrame
print(clean_df)

In [76]:
## If you want to modify the DataFrame in-place and reset the index
clean_df.drop_duplicates(subset='text_clean', inplace=True)
len(clean_df)

20701

In [78]:
output_file = "/content/drive/MyDrive/Subasish(2023)_Twitter/Tutorial/unique_clean_15minutecity.xlsx"
clean_df.to_excel(output_file, index=False)

In [79]:
len(clean_df)

20701

In [80]:
clean_df.head(60)

,verif_ID,tweet_id,text,sourcetweet_type,sourcetweet_id,text1,user_username,created_at,date,conversation_id,...,user_following_count,sourcetweet_text,sourcetweet_lang,sourcetweet_author_id,mention,hashtag,links,emoji,text_clean,original_language
0,a00000000,1024126649057590,RT @erinriediger: Another example of how cities can be re-designed for people even if we've made urban planning mistakes in the past. The c…,retweeted,1.023349e+15,Another example of how cities can be re-designed for people even if we've made urban planning mistakes in the past. The comfortable pedestrian environment now co-exists with car lanes. #PortageAndMain #TeamOpen #wpgpoli #citiesforpeople #walkablecity https://t.co/ZDixI3ku0m,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"PortageAndMain, TeamOpen, wpgpoli, citiesforpeople, walkablecity",https://t.co/ZDixI3ku0m,NaN,another example of how cities can be redesigned for people even if weve made urban planning mistakes in the past the comfortable pedestrian environment now coexists with car lanes,NaN
1,a00000001,1024038782842986,#KNOWYOUREDMO\nAny idea where this spot is in town? Take a guess.\n\n#guessinggame #knowyourcity #deepedmonton #yeg #edmo #edmonton #walkyeg #yegwalk #walkable #walkableyeg #walkableedmonton #walking #walkability #alberta #walkablecity https://t.co/fRbetWbNsa,NaN,0.000000e+00,#KNOWYOUREDMO\nAny idea where this spot is in town? Take a guess.\n\n#guessinggame #knowyourcity #deepedmonton #yeg #edmo #edmonton #walkyeg #yegwalk #walkable #walkableyeg #walkableedmonton #walking #walkability #alberta #walkablecity https://t.co/fRbetWbNsa,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"KNOWYOUREDMO, guessinggame, knowyourcity, deepedmonton, yeg, edmo, edmonton, walkyeg, yegwalk, walkable, walkableyeg, walkableedmonton, walking, walkability, alberta, walkablecity",https://t.co/fRbetWbNsa,NaN,any idea where this spot is in town take a guess,NaN
2,a00000002,1023921603334287,@JaniceLukes @OlyBackstrom @pensato @adamdooley If you support @880CitiesOrg and accessibility does that mean you support opening the intersection to pedestrians? Even if you think the public consult process was lacking it seems your views would be supportive of a #walkablecity ?,NaN,0.000000e+00,@JaniceLukes @OlyBackstrom @pensato @adamdooley If you support @880CitiesOrg and accessibility does that mean you support opening the intersection to pedestrians? Even if you think the public consult process was lacking it seems your views would be supportive of a #walkablecity ?,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"JaniceLukes, OlyBackstrom, pensato, adamdooley, 880CitiesOrg",walkablecity,NaN,NaN,if you support and accessibility does that mean you support opening the intersection to pedestrians even if you think the public consult process was lacking it seems your views would be supportive of a,NaN
3,a00000003,1023920796153069,"Remove barriers, coalition says https://t.co/75pCVt9Lt3 \n#walkablecity #safecity #teamopen #portageandmain",NaN,0.000000e+00,"Remove barriers, coalition says https://t.co/75pCVt9Lt3 \n#walkablecity #safecity #teamopen #portageandmain",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"walkablecity, safecity, teamopen",https://t.co/75pCVt9Lt3,NaN,remove barriers coalition says,NaN
4,a00000004,1023700403945439,"Everyday pride, Vancouver BC. \n.\n.\n.\n.\n#pride #pride🌈 #vancouver #vancouverbc #vancouverpride #helmckenstreet #burrard #burrardstreet #aptasustainability #apta #walkable #walkablecity… https://t.co/viYI80saXO",NaN,0.000000e+00,"Everyday pride, Vancouver BC. \n.\n.\n.\n.\n#pride #pride🌈 #vancouver #vancouverbc #vancouverpride #helmckenstreet #burrard #burrardstreet #aptasustainability #apta #walkable #walkablecity… https://t.co/viYI80saXO",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"pride, vancouver, vancouverbc, vancouverpride, helmckenstreet, burrard, burrardstreet, aptasustainability, apta, walkable",https://t.co/viYI80saXO,🌈,everyday pride vancouver bc,NaN
5,a00000005,1023690466607489,My favorite part of walking the